In [12]:
# 安装必要的库
!pip install streamlink
!pip install ultralytics

import streamlink
import cv2
from ultralytics import YOLO
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# 获取用户输入想要检测的物体名称
input_box = widgets.Text(placeholder='输入你想查询的物体名称')
button = widgets.Button(description='开始检测')
display(input_box, button)

# 回调函数，开始视频处理
def start_detection(b):
    target_object = input_box.value.lower()
    if not target_object:
        print("请先输入要查询的物体名称！")
        return

    # 获取直播流 URL
    youtube_url = "https://www.youtube.com/live/_KVWehizoNU"
    streams = streamlink.streams(youtube_url)

    # 选择合适的流质量
    if "best" in streams:
        stream_url = streams["best"].url
    else:
        print("未找到合适的流，检查直播源")
        return

    # 打开视频流
    cap = cv2.VideoCapture(stream_url)
    if not cap.isOpened():
        print("无法打开视频流，请检查 URL 是否正确")
        return

    # 加载预训练的 YOLOv8 模型
    model = YOLO('yolov8n.pt')

    # 处理视频流
    while True:
        ret, frame = cap.read()
        if not ret:
            print("无法获取视频帧，视频结束或读取失败")
            break

        # 使用 YOLO 模型进行目标检测
        results = model(frame)

        # 保存当前帧的目标物体数量
        detected_count = 0
        for result in results:
            for box in result.boxes:
                # 获取检测的类别
                class_id = int(box.cls[0])
                class_name = model.names[class_id].lower()

                # 累加指定类别的计数
                if class_name == target_object:
                    detected_count += 1

        # 打印检测结果
        print(f"在当前帧中，检测到的 {target_object} 数量为：{detected_count}")

        # 将帧从 BGR 转为 RGB，以便在 Jupyter 中显示
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 使用 Matplotlib 显示图像（适用于 Jupyter 环境）
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 6))
        plt.imshow(frame_rgb)
        plt.axis('off')
        plt.show()

        # 等待用户确认继续（为了演示效果，等待 2 秒）
        import time
        time.sleep(2)

        # 如果需要退出，可以手动停止循环
        break

    # 释放视频捕获对象
    cap.release()
    cv2.destroyAllWindows()

# 将按钮点击事件绑定到回调函数
button.on_click(start_detection)


Text(value='', placeholder='输入你想查询的物体名称')

Button(description='开始检测', style=ButtonStyle())